In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import PY

In [3]:
# import cf.management as mng
from cafle import management_actualcase1 as mng1

In [4]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction
* Financial model for a development of a logistic center

# Initial Setting

In [5]:
class Assumption:
    def __init__(self):
        pass
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
astn = Assumption()

In [6]:
class Assumption_sales:
    def __init__(self):
        self.prdtname = []
        self.mtrx = DataFrame()
        self.csh = {}
        self.ctrt = {}
        self.ctrt_ary = {}
        
    def iptctrt(self, prdt_name, tmpary):
        tmp_idx = [x[0] for x in tmpary]
        tmp_locno = [idx.sales.locno(x[0]) for x in tmpary]
        tmp_rate = [x[1] for x in tmpary]
        self.ctrt_ary[prdt_name] = tmpary
        self.ctrt[prdt_name] = DataFrame({'locno':tmp_locno,
                                          'rate':tmp_rate},
                                          index=tmp_idx)
        self.ctrt[prdt_name]['amt'] = self.ctrt[prdt_name]['rate'] \
                                      * self.mtrx.amt[prdt_name]
        self.ctrt[prdt_name]['odr'] = list(range(len(tmpary)))
        self.ctrt[prdt_name] = self.ctrt[prdt_name][['odr', 'locno', 'rate', 'amt']]
        
    def print_ctrt(self):
        print("Sales Contract Assumption")
        for key, val in self.ctrt.items():
            print("\n\n"+key)
            display(R(val))

    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
            
astns = Assumption_sales()

In [7]:
class Assumption_costs:
    def __init__(self):
        self.dct = {}
        self.amt = {}
        self.byname = {}
    
    def __getattr__(self, attr):
        return self.__dict__[attr]
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
astnc = Assumption_costs()

# Input Index Data

In [8]:
# Set period index
# prjt index : total project periods, and project base index
# cstrn index : construction index
# loan index : loan period index
# sales index : sales period index

tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-11',   27],
           ['cstrn', '2021-12',   20],
           ['loan',  '2021-12',   24],
           ['sales', '2021-12',   24]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq = 'M')

# Input Financing Condition Data

In [9]:
# Equity condition
tmp_col =  ['title',  'amt_ntnl', 'amt_ntnl_once']
tmp_ary = [['equity',    1_300,        1_300]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx,
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_ntnl_once = astn.equity.amt_ntnl_once[0])

In [10]:
tmp_col = ['title', 'amt_ntnl', 'amt_ntnl_once', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',     55_000,       10_000,        0.02,      0.040],
           ['trb',     16_500,       16_500,        0.05,      0.060],
           ['trc',      9_000,        9_000,        0.08,      0.080]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_ntnl_once = astn.loan.amt_ntnl_once,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)